In [32]:
import random
import Crypto
import math
import sys

class Server:
 
    def __init__(self, ppkc,x_ctext, y_ctext):
        self.ppkc = ppkc
        self.x_ctext = x_ctext
        self.y_ctext = y_ctext
        self.l = 16
        self.expl = int(2**self.l)
        
    def server_one(self):
        z_ctext = [0,0]
        d_ctext = [0,0]
        z_ctext0 = [0,0]
        r_ptext = random.randint(0,expl-1)
        r_ctext = pailliermodencrypt(r_ptext,self.ppkc)
        z_ctext0[1] = ((pailliermodencrypt(self.expl,self.ppkc))[1]*self.x_ctext[1])%(self.ppkc[0]**2)
        z_ctext[1] =(z_ctext0[1]*pow(self.y_ctext[1],-1,self.ppkc[0]**2))%(self.ppkc[0]**2)
        z_ctext0[0] = ((pailliermodencrypt(self.expl,self.ppkc))[0]*self.x_ctext[0])%(self.ppkc[0]**2)
        z_ctext[0] = (z_ctext0[0]*pow(self.y_ctext[0],-1,self.ppkc[0]**2))%(self.ppkc[0]**2)
        d_ctext[1] = (z_ctext[1]*r_ctext[1])%(self.ppkc[0]**2)
        d_ctext[0] = (z_ctext[0]*r_ctext[0])%(self.ppkc[0]**2)
        cap_r_ptext = r_ptext%self.expl
        return r_ptext,d_ctext,cap_r_ptext,z_ctext
    
    def server_two(self, cap_r_ptext, z_ctext, lambda_ctext, cap_d_ctext):
        b_ctext = [0,0]
        b_ctext1 = [0,0]
        b_ctext2 = [0,0]
        cap_r_ctext = int(pailliermodencrypt(cap_r_ptext, self.ppkc)[1])
        b = pow(lambda_ctext,self.expl,self.ppkc[0]**2)
        c = pow(2, -self.l)
        b_ctext1[1] = (z_ctext[1]*pow(cap_d_ctext[1], -1, self.ppkc[0]**2))%(self.ppkc[0]**2)
        b_ctext1[0] = (z_ctext[0]*pow(cap_d_ctext[0], -1, self.ppkc[0]**2))%(self.ppkc[0]**2)
        b_ctext2[1] = (b_ctext1[1] * cap_r_ctext)%(self.ppkc[0]**2)
        b_ctext2[0] = (b_ctext1[0] * cap_r_ctext)%(self.ppkc[0]**2)
        b_ctext[1] = (b_ctext2[1] * pow(b, -1, self.ppkc[0]**2))%(self.ppkc[0]**2)
        b_ctext[0] = (b_ctext2[0] * pow(b, -1, self.ppkc[0]**2))%(self.ppkc[0]**2)
        return b_ctext
    
class Client:

    def __init__(self,ppkc,pskc,l,expl):
        self.ppkc = ppkc
        self.pskc = pskc
        self.l = l
        self.expl = expl
    
    def client_bitmin_one(self,d_ctext,cap_r_ptext):
        d_ptext = int(pailliermoddecrypt(d_ctext,self.ppkc,self.pskc))
        cap_d_ptext = d_ptext%self.expl
        cap_d_ctext = [0,0]
        cap_d_ctext[1] = int(pailliermodencrypt(cap_d_ptext,self.ppkc)[1])
        cap_d_ctext[0] = int(pailliermodencrypt(cap_d_ptext,self.ppkc)[0])
        lambda_ptext = int(cap_d_ptext < cap_r_ptext)
        lambda_ctext = pailliermodencrypt(lambda_ptext,self.ppkc)
        return d_ptext, cap_d_ctext, lambda_ctext

def modInverse(a, p):
    maxiter = int(1.0e+100)

    if a == 0:
        raise ValueError('0 has no inverse mod %d' % p)
    r = int(a)
    d = 1
    for i in range(min(p, maxiter)):
        d = ((int(p // r) + 1) * d) % p
        r = int((d * a) % p)
        if r == 1:
            break
    else:
        raise ValueError('%d has no inverse mod %d' % (a, p))
    return d
         
    
def paillierkeygen():
    while True:
        p = random.getrandbits(24)
        q = random.getrandbits(24)
        if math.gcd(p*q,(p-1)*(q-1))==1 and isPrime(p) and isPrime(q):
            break
    n = p*q
    print (p,q,n)

    g = n + 1
    phi = (p-1)*(q-1)
    mu = pow(phi,-1, n)
    pkc = [n,g]
    skc = [phi,mu]
    return pkc,skc


def pailliermodkeygen():
    while True:
        a = random.getrandbits(20);
        
        if isPrime(a):
            p = 2*a+1
            if isPrime(p):
                print(a)
                print(p)
                break
    while True:
        b = random.getrandbits(20);
        q = 2*b+1;
        if isPrime(b) and isPrime(q):
            print(b)
            print(q)
            break
    
    n = p*q;
    x = random.randint(1, math.floor(n*n/2));
    
    while 1:
        k = random.randint(1, n*n);
        if (math.gcd(k,n*n) == 1):
            break
    
    g = pow(k,2*n,n)
    print(g)
    
    h = pow(g,x,n*n)
    pkc = [n,g,h]
    
    skc = x
    sskc = [p,q]
    return pkc, skc, sskc

def paillierencrypt(ptext, pkc):
    n = pkc[0]
    while 1:
        r = random.getrandbits(17)
        if r < n and r > 0 and math.gcd(r,n) == 1 and isPrime(r):
            break
    g = pkc[1]
    out = (pow(g,ptext,n*n)*(pow(r,n,n*n)))%(n*n)
    return out

def pailliermodencrypt(ptext,pkc):
    r = random.randint(1,math.floor(pkc[0]/4))
    t1 = pow(pkc[1],r,pkc[0]**2)
    t2 = (pow(pkc[2],r,pkc[0]**2)*((1 + ptext*pkc[0])%(pkc[0]**2)))%(pkc[0]**2)
    ctext = [t1,t2]
    return ctext
    
def paillierdecrypt(ctext,pkc,skc):
    n = pkc[0]
    x = pow(ctext,skc[0],n*n) - 1
    out = ((x//n)*skc[1]) % n
    return out

def pailliermoddecrypt(ctext,pkc,skc):
    l = pow(ctext[0], skc, pkc[0]**2)
    k = pow(l,-1,pkc[0]**2)
    u = (ctext[1]*k)%(pkc[0]**2)
    print(u)
    print(u%pkc[0])
    ptext = ((u - 1)/pkc[0])%(pkc[0]**2)
    return ptext


def isPrime(num):
    if num > 1:
        for i in range(2,num):
            if (num % i) == 0:
                return False
                break
        else:
            return True
    else:
        return False
    


In [33]:
mpkc, mskc, msskc = pailliermodkeygen()

756149
1512299
73823
147647
222552327713


In [34]:
ptext1 = 67
ctext1 = pailliermodencrypt(ptext1,mpkc)
print(ctext1)

[11952444277153118455394, 6124781696638461329905]


In [35]:
ptext2 = 26
ctext2 = pailliermodencrypt(ptext2,mpkc)

In [36]:
ptext3 = 65
ctext3 = pailliermodencrypt(ptext3,mpkc)

In [37]:
ctext = [0,0]
ctext0 = [0,0]

ctext[1]= (ctext1[1] * pow(ctext2[1],-1,mpkc[0]**2) )%(mpkc[0]**2)
ctext0[1] = (ctext[1] * ctext3[1]) % (mpkc[0] ** 2)
ctext[0]= (ctext1[0] * pow(ctext2[0],-1,mpkc[0]**2))%(mpkc[0]**2)
ctext0[0] = (ctext[0] * ctext3[0]) % (mpkc[0]**2)

In [38]:
x = pailliermoddecrypt(ctext0,mpkc,mskc)

23668359508019
1


In [39]:
x%mpkc[0]

106.0

In [40]:
pskc

NameError: name 'pskc' is not defined

In [41]:
#x_ptext = random.getrandbits(16)
#y_ptext = random.getrandbits(16)

In [42]:
x_ptext = 100 - 92
y_ptext = 3

In [43]:
x_ctext = pailliermodencrypt(x_ptext,mpkc)
y_ctext = pailliermodencrypt(y_ptext,mpkc)

In [44]:
x_ctext

[21989134185600873184252, 42454156496198085280507]

In [45]:
s = Server(mpkc,x_ctext,y_ctext)

In [46]:
l = s.l
expl = s.expl

In [47]:
z_ptext = (expl + x_ptext - y_ptext)%mpkc[0]
print(z_ptext)

65541


In [48]:
c = Client(mpkc,mskc,l,expl)

In [49]:
r_ptext,d_ctext,cap_r_ptext, z_ctext = s.server_one()

In [50]:
(r_ptext+z_ptext)%mpkc[0]

75283

In [51]:
pailliermoddecrypt(z_ctext,mpkc,mskc)

18021711334591620624304
123024358582


80711187474.55096

In [31]:
x_ptext.bit_length()

4

In [52]:
(pailliermodencrypt(s.expl,mpkc)[1] * s.x_ctext[1] ) % (mpkc[0]**2)

30476739101239848248838

In [53]:
(30476739101239848248838* pow(y_ctext[1],-1,s.ppkc[0]**2)) % (mpkc[0]**2)

25518151957805307273441

In [54]:
(pailliermodencrypt(s.expl,mpkc)[0] * s.x_ctext[0] ) % (mpkc[0]**2)

15242274743353040489924

In [55]:
(15242274743353040489924* pow(y_ctext[0],-1,s.ppkc[0]**2))% (mpkc[0]**2)

1393293223089498974248

In [56]:
pailliermoddecrypt([1393293223089498974248,25518151957805307273441],mpkc, mskc)

12875942714903835636646
147344289274


57665590524.65989

In [76]:
d_ptext, cap_d_ctext, lambda_ctext= c.client_bitmin_one(d_ctext,cap_r_ptext)

782465549277756162578057
4520670712


In [77]:
b_ctext = s.server_two(cap_r_ptext, z_ctext, lambda_ctext, cap_d_ctext)

TypeError: unsupported operand type(s) for pow(): 'list', 'int', 'int'

In [78]:
b_ctext

NameError: name 'b_ctext' is not defined

In [136]:
b_ptext = pailliermoddecrypt(b_ctext, mpkc, mskc)

1517613036478887251
112094020


In [137]:
b_ptext

1194964123.0882626

In [138]:
x_ptext<y_ptext

False

In [139]:
r_ptext

37338

In [126]:
d_ptext

225

In [127]:
z_ptext

14

In [128]:
(d_ptext-r_ptext)%expl

4494

In [129]:
(d_ptext%expl-r_ptext%expl)+0

-61042

In [130]:
z_ptext-13424

-13410

In [29]:
65536*2

131072

In [30]:
2**17

131072